# Práctica: Combinación de DataFrames (Merge, Concat y Join)

## 1. Preparación de los Datos
Crearemos dos DataFrames principales:
1. `estudiantes`: Lista maestra de alumnos.
2. `notas`: Registro de calificaciones de exámenes.
**Nota:** Introduciremos intencionalmente IDs que no coinciden en ambas tablas para ver cómo se comportan los `joins`.

In [ ]:
import pandas as pd
import numpy as np

# Tabla 1: Información de Estudiantes
data_estudiantes = {
    'id_estudiante': [101, 102, 103, 104, 105],
    'nombre': ['Ana', 'Luis', 'Carlos', 'Sofia', 'Jorge'],
    'carrera': ['Ingeniería', 'Derecho', 'Ingeniería', 'Medicina', 'Arquitectura']
}

# Tabla 2: Calificaciones (Fíjate que falta el estudiante 105 y sobra el 108)
data_notas = {
    'id_estudiante': [101, 102, 103, 108], # 108 no existe en estudiantes
    'materia': ['Cálculo', 'Penal', 'Física', 'Anatomía'],
    'calificacion': [85, 90, 78, 95]
}

df_estudiantes = pd.DataFrame(data_estudiantes)
df_notas = pd.DataFrame(data_notas)

print("--- Tabla: Estudiantes ---")
display(df_estudiantes)
print("\n--- Tabla: Notas ---")
display(df_notas)

## 2. Uso de `merge()`
Es la función más flexible. Relaciona filas basándose en una columna común (clave).

### 2.1 Inner Join (Intersección)
Solo trae los registros que existen en **ambas** tablas.
- **Caso:** ¿Qué estudiantes tienen nota registrada?
- **Resultado esperado:** Jorge (105) y el desconocido (108) desaparecen.

In [ ]:
# INNER JOIN: La intersección
# on='id_estudiante' indica la columna clave para cruzar
df_inner = pd.merge(df_estudiantes, df_notas, on='id_estudiante', how='inner')

print("--- Inner Join (Solo coincidencias) ---")
display(df_inner)

### 2.2 Left Join (Mantener Izquierda)
Mantiene **todas** las filas de la tabla izquierda (`estudiantes`), y pega la información de la derecha (`notas`) donde haya coincidencia. Si no hay nota, pone `NaN`.
- **Caso:** Quiero una lista de *todos* mis estudiantes, con sus notas si las tienen.

In [ ]:
# LEFT JOIN: Prioridad a la tabla izquierda (Estudiantes)
df_left = pd.merge(df_estudiantes, df_notas, on='id_estudiante', how='left')

print("--- Left Join (Todos los estudiantes) ---")
display(df_left)
# Nota cómo Jorge (105) aparece con NaN en materia y calificacion

## 3. Uso de `pd.concat()`
Se usa para "pegar" DataFrames.
* **Vertical (`axis=0`):** Pegar uno debajo del otro (más filas).
* **Horizontal (`axis=1`):** Pegar uno al lado del otro (más columnas).

In [ ]:
# CASO 1: Concatenación Vertical (Más filas)
# Imaginemos que llegan datos de nuevos inscritos
nuevos_inscritos = pd.DataFrame({
    'id_estudiante': [106, 107],
    'nombre': ['Elena', 'Pedro'],
    'carrera': ['Diseño', 'Sistemas']
})

# Pegamos los nuevos debajo de los originales
# ignore_index=True resetea los índices 0,1,2...
df_total_alumnos = pd.concat([df_estudiantes, nuevos_inscritos], axis=0, ignore_index=True)

print("--- Concat Vertical (Lista completa) ---")
display(df_total_alumnos)

In [ ]:
# CASO 2: Concatenación Horizontal (Más columnas)
# Tenemos datos extra que sabemos que están en el MISMO ORDEN de filas
info_contacto = pd.DataFrame({
    'email': ['ana@school.edu', 'luis@school.edu', 'carlos@school.edu', 'sofia@school.edu', 'jorge@school.edu'],
    'telefono': ['555-1111', '555-2222', '555-3333', '555-4444', '555-5555']
})

# Pegamos al lado (axis=1)
df_detallado = pd.concat([df_estudiantes, info_contacto], axis=1)

print("--- Concat Horizontal (Enriquecimiento) ---")
display(df_detallado)

## 4. Uso de `join()` y `set_index()`
`join()` está optimizado para trabajar sobre los **índices** (las etiquetas de las filas) en lugar de columnas. Es muy rápido y limpio.

Para usarlo bien, primero convertimos la columna ID en el Índice del DataFrame.

In [ ]:
# 1. Preparamos los DataFrames estableciendo el índice
# Ahora 'id_estudiante' deja de ser columna y pasa a ser el índice (negrita a la izquierda)
df_est_index = df_estudiantes.set_index('id_estudiante')
df_notas_index = df_notas.set_index('id_estudiante')

print("--- Vista de DataFrames indexados ---")
display(df_est_index.head(2))

# 2. Ejecutamos el JOIN
# Por defecto join() hace un 'left join' usando los índices
df_unido_indices = df_est_index.join(df_notas_index, lsuffix='_izq', rsuffix='_der')

print("\n--- Resultado de df.join() ---")
display(df_unido_indices)

## 📌 Interpretación de Resultados
1. **Merge:** Es ideal cuando tus claves de unión son columnas normales. El `Left Join` es vital para detectar datos faltantes (ej. estudiantes sin nota).
2. **Concat:** Es la herramienta para unir reportes mensuales (vertical) o agregar características desordenadas (horizontal).
3. **Join:** Es la versión optimizada de merge cuando tus datos ya están estructurados por índices.